# Data Cleaning with Pandas /Matching Strings by Similarity using Levenshtein distance

In [ ]:
In this project, we're going to resolve a very tough problem, which is identifying similar strings that are not completely equal. It's a very common and challenging data-cleaning task, so let's get right into it!

We have two CSVs containing company names. The problem is that the company names in these two CSVs don't match. Both have a single column, but if you explore them, you'll immediately spot the anomalies. For example, here are the first 5 rows of the first CSV (companies_1.csv):

CLIENT
"Adobe Systems, Inc."
Adventist Health
AECOM                                   <------ CHECK THIS OUT
Aerojet Rockedyne Holdings (GenCorp)
Alameda-Contra Costa Transit District
Alaska Community Foundation
Alaska Retirement Management Board
"Alexander & Baldwin, Inc."
"Allergan, Inc."

And here's for the second CSV (companies_2.csv):

Firm Name
"AAA Northern California, Nevada & Utah Auto Exchange"
ACCO Engineered Systems
Adams County Retirement Plan
"Adidas America, Inc."
"Adobe Systems, Inc."
"Advanced Micro Devices, Inc."
AECOM Technology Corporation             <------ CHECK THIS OUT
Aera Energy LLC
"Aerojet Rocketdyne Holdings, Inc."
You can see that the company AECOM appears as AECOM in CSV1, but as AECOM Technology Corporation in CSV2.

To do so, we're going to use an external library called thefuzz that contains functionality to compute the Levenshtein distance between two strings. More on this later.

Your job will be cleaning this data and matching the columns in both CSVs. Let's get started.

Data Preprocessing
Before we can compare the company names, we need to read the data and align the companies for comparison. To do so, we'll need to use the itertools.product function to create the "cartesian product" between the two CSVs. Very briefly, the Cartesian Product of two collections A and B, returns a combination of ALL the elements in both A and B.

Preview
Example:

>>> A = ["Apple", "Alphabet", "Microsoft"]
>>> B = ["MSFT", "Alphabet/Google", "Apple inc."]

>>> list(itertools.product(A, B))
[('Apple', 'MSFT'),
 ('Apple', 'Alphabet/Google'),
 ('Apple', 'Apple inc.'),
 ('Alphabet', 'MSFT'),
 ('Alphabet', 'Alphabet/Google'),
 ('Alphabet', 'Apple inc.'),
 ('Microsoft', 'MSFT'),
 ('Microsoft', 'Alphabet/Google'),
 ('Microsoft', 'Apple inc.')]
As you can see, every element in A is matched with every element in B.


In [ ]:
import itertools
import pandas as pd

In [ ]:
# The new library!
from thefuzz import fuzz, process

df1 = pd.read_csv('companies_1.csv')
df2 = pd.read_csv('companies_2.csv')

In [ ]:
### Data Preprocessing

##### 1. Create the `df` dataframe containing the product of the two CSVs

We have already read the 2 CSVs into the df1 and df2 variables. Now, use the itertools.product method to create a resulting dataframe df that will contain the product of the two CSVs. The columns should be named CSV 1 and CSV 2.

csv_1= df1["CLIENT"].values
csv_2= df2["Firm Name"].values

companies= itertools.product(csv_1, csv_2)

df = pd.DataFrame(companies,columns=["CSV 1","CSV 2"])

df.head()

### Calculating the Levenshtein distance

Now, we will learn how to calculate the Levenshtein distance between two strings. Here we will user `partial_ratio` function from the `fuzz` module to compute the "ratio" between two strings. The result is a number between `0` and `100`, with `100` indicating a "perfect" match. Please note that `partial_ratio` gives ratio of the shortest string length to the longest string length. For example, if the first string is `ABC` and the second string is `ABDC`, then the ratio will be `4/5 = 0.80`. 

We're going to use the partial_ratio function from the fuzz module to compute the "distance" between two strings. The result is a number between 0 and 100, with 100 indicating a "perfect" match. Please keep in mind that this method is not perfect, as shown in the examples about Microsoft in the notebook.

fuzz.partial_ratio("Apple", "Apple Inc.")

fuzz.partial_ratio("Microsoft", "Apple Inc.")

fuzz.partial_ratio("Microsoft", "MSFT")

If we have list of strings, we can calculate the Levenshtein distance between each pair of strings in the list.

A = ["Apple", "Alphabet", "Microsoft"]
B = ["MSFT", "Alphabet/Google", "Apple inc."]

Below, we combined the two list `A` and `B` into a list of tuples `companies` using `product` function from `itertools` module. 

Then, we calculated the partial ratio for each pair of strings in the list `companies` using `partial_ratio` function from `fuzz`.

companies = list(itertools.product(A, B))
companies

for c1, c2 in companies:
    ratio = fuzz.partial_ratio(c1, c2)
    print(f"{c1} > {c2}: {ratio}")

You will see the greater the ratio, the more similar the strings are.

##### 2. Create a new column `Ratio Score` that contains the distance for all the rows in `df`

scores = [fuzz.partial_ratio(c1, c2) for c1, c2 in df.values]

df['Ratio Score'] = scores

df['Ratio Score'] = df.apply(lambda row: fuzz.partial_ratio(row['CSV 1'], row['CSV 2']), axis=1)


##### 3. How many rows have a Ratio score of `90` or more?

df.query("`Ratio Score` >= 90").shape

len (df.loc[df['Ratio Score'] >= 90])

##### 4. What's the corresponding company in CSV2 to `AECOM` in CSV1?

We saw that in CSV 1 there's a company AECOM, what's the corresponding value in CSV 2? The higher the Ratio Score, the more likely it is to be a match. So, use the Ratio Score greater than 90 to filter the results.

len(df["Ratio Score"] >90)

 df.loc[(df["CSV 1"]=="AECOM") & (df["Ratio Score"] >90)]

df.query("`CSV 1` == 'AECOM' and `Ratio Score` > 90")


##### 5. What's the corresponding CSV2 company of *Starbucks*?

df.loc[(df["CSV 1"]=="Starbucks") & (df["Ratio Score"] >90)]

df.query("`CSV 1` == 'Starbucks' and `Ratio Score` > 90")




##### 6. Is there a matching company for `Pinnacle West Capital Corporation`?

6.Is there a matching company for Pinnacle West Capital Corporation in column CSV 2?

Column CSV 1 contains Pinnacle West Capital Corporation, name a matching company in column CSV 2 with a ratio score more than 90?


RAND Corporation
Ball Corporation
Avista Corporation
Rogers Corporation

df.loc[(df["CSV 1"]=="Pinnacle West Capital Corporation") & (df["Ratio Score"] > 90)]

df.query("`CSV 1` == 'Pinnacle West Capital Corporation' and `Ratio Score` > 90")



##### 7. How many matching companies are there for `County of Los Angeles Deferred Compensation Program`?

CSV 1 contains the County of Los Angeles Deferred Compensation Program. How many matching companies seem to be in CSV 2?

df.loc[(df["CSV 1"]=="County of Los Angeles Deferred Compensation Program") & (df["Ratio Score"] > 90)]



##### 8. Is there a matching company for `The Queens Health Systems`?

CSV 1 contains The Queens Health Systems, is there a matching in CSV 2?

df.loc[(df["CSV 1"]=="The Queens Health Systems") & (df["Ratio Score"] > 90)]



### The End!